# Lesion classification with fractal neural networks

__TL;DR__

We create a fractal neural network with an Inception ResNet V2 in its core and study its performance for lesion classification.

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from random import shuffle
import tensorflow_hub as hub
import tensorflow_addons as tfa
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from IPython.display import Image, display
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

Remove excessive logging. 

In [ ]:
tf.get_logger().setLevel('ERROR')

# Melanoma

__Melanoma__, also redundantly known as __malignant melanoma__, is a type of skin cancer that develops from the pigment-producing cells known as melanocytes. Melanomas typically occur in the skin, but may rarely occur in the mouth, intestines, or eye (uveal melanoma). In women, they most commonly occur on the legs, while in men, they most commonly occur on the back. About 25% of melanomas develop from moles. Changes in a mole that can indicate melanoma include an increase in size, irregular edges, change in color, itchiness, or skin breakdown.

![melanoma image](../assets/melanoma.jpg)

<div style="text-align: center; font-weight: bold">Pic.1. A melanoma of approximately 2.5 cm (1 in) by 1.5 cm (0.6 in)</div>

The primary cause of melanoma is ultraviolet light (UV) exposure in those with low levels of the skin pigment melanin. The UV light may be from the sun or other sources, such as tanning devices. Those with many moles, a history of affected family members, and poor immune function are at greater risk. A number of rare genetic conditions, such as xeroderma pigmentosum, also increase the risk. Diagnosis is by biopsy and analysis of any skin lesion that has signs of being potentially cancerous.

Melanoma is the most dangerous type of skin cancer. Globally, in 2012, it newly occurred in 232,000 people. In 2015, 3.1 million people had active disease, which resulted in 59,800 deaths. Australia and New Zealand have the highest rates of melanoma in the world. High rates also occur in Northern Europe and North America, while it is less common in Asia, Africa, and Latin America. In the United States, melanoma occurs about 1.6 times more often in men than women. Melanoma has become more common since the 1960s in areas mostly populated by people of European descent.

# Fractal neural network

We propose an ensemble model based on handcrafted fractal features and deep learning that consists of combining the classification of two CNNs by applying the sum rule. We apply feature extraction to obtain 300 fractal features from different
dermoscopy datasets. These features are reshaped into a 10 × 10 × 3 matrix to compose an artificial image that
is given as input to the first CNN. The second CNN model receives as input the correspondent original image.

![CNN image](../assets/fnn.png)

<div style="text-align: center; font-weight: bold">Pic.2. Overview of the proposed FNN model.</div>

If you want to learn more about fractal neural networks, read [here](https://www.sciencedirect.com/science/article/abs/pii/S0957417420308563).

## Dividing images into patches.

According to the acticle:
> One of the approaches available in the literature for multiscale analysis is the gliding-box algorithm (Ivanovici & Richard, 2011). The main advantage of this approach is that it can be applied on datasets containing images with different resolutions since the output features are given in relation to the scale instead of being absolute values. This algorithm consists in placing a box $\beta_{i}$ sized $𝐿 × 𝐿$ on the left superior corner of the image, wherein 𝐿 is given in pixels. This box glides through the image, one column and then one row at a time. After reaching the end of the image, the box is repositioned at the starting point and the value of 𝐿 is increased by 2.

The gliding-box method will not be used since it consumes too much RAM. We'll employ a box-counting approach, which basically means we'll partition the images into non-overlapping chunks.

In [ ]:
class Patchify(tf.keras.layers.Layer):
    def __init__(self, patch_size):
        super(Patchify, self).__init__()
        
        self.patch_size = patch_size
        
    def call(self, inputs):
        outputs = tf.image.extract_patches(
            inputs,
            sizes=(1, self.patch_size, self.patch_size, 1),
            strides=(1, self.patch_size, self.patch_size, 1),
            rates=(1, 1, 1, 1),
            padding='SAME'
        )
        
        _, rows, cols, _ = tf.unstack(tf.shape(outputs))
        outputs = tf.reshape(outputs, shape=(-1, rows * cols, self.patch_size, self.patch_size, 3))
        
        return outputs

## Creating an array of binary values from image patches using the Chebyshev colour distance function applied to the patch centre and each pixel.

According to the article:
> For each time the box $\beta_{i}$ is moved, a multidimensional analysis of colour similarity is performed for every pixel inside it. This is done by assigning the centre pixel to a vector $𝑓_{c} = 𝑟_{c}, 𝑔_{c}, 𝑏_{c}$, where $𝑟_{c}, 𝑔_{c}$ and $𝑏_{c}$ correspond to the colour intensities for each of the RGB colour channels of given pixel. The other pixels in the box are assigned to a vector $𝑓_{i} = 𝑟_{i}, 𝑔_{i}, 𝑏_{i}$ and compared to the centre pixel by calculating a colour distance $\Delta$. On the proposed approach, the Chebyshev ($\Delta_{h}$) ...

The following equation is used to compute the Chebyshev distance.

$$
\Delta_{h} = max(|f_{i}(k_{i}) - f_{c}(k_{c})|), k \in r, g, b. 
$$ 

In [ ]:
class Chebyshev(tf.keras.layers.Layer):
    def __init__(self):
        super(Chebyshev, self).__init__()
        
    def call(self, inputs):
        batch_size, patch_number, patch_size, patch_size, channels = tf.unstack(tf.shape(inputs))
        outputs = tf.reshape(inputs, shape=(-1, patch_number, patch_size, channels))    
        
        centers = tf.image.resize_with_crop_or_pad(outputs, 1, 1)

        outputs = tf.math.subtract(outputs, centers)
        outputs = tf.math.abs(outputs)
        outputs = tf.math.reduce_max(outputs, axis=3)
        outputs = tf.math.less_equal(outputs, tf.cast(patch_size, dtype=tf.float32))
        outputs = tf.cast(outputs, dtype=tf.int32)
        outputs = tf.reshape(outputs, shape=(-1, patch_number, patch_size, patch_size))
        
        return outputs

## Creating an array of binary values from image patches using the Euclidean colour distance function applied to the patch centre and each pixel.

According to the article:
> For each time the box $\beta_{i}$ is moved, a multidimensional analysis of colour similarity is performed for every pixel inside it. This is done by assigning the centre pixel to a vector $𝑓_{c} = 𝑟_{c}, 𝑔_{c}, 𝑏_{c}$, where $𝑟_{c}, 𝑔_{c}$ and $𝑏_{c}$ correspond to the colour intensities for each of the RGB colour channels of given pixel. The other pixels in the box are assigned to a vector $𝑓_{i} = 𝑟_{i}, 𝑔_{i}, 𝑏_{i}$ and compared to the centre pixel by calculating a colour distance $\Delta$. On the proposed approach, ... the Euclidean ($\Delta_{e}$) ...

$$
\Delta_{e} = \sqrt{\sum_{k} (f_{i}(k_{i}) - f_{c}(k_{c}))^2}, k \in r, g, b
$$

In [ ]:
class Euclidean(tf.keras.layers.Layer):
    def __init__(self):
        super(Euclidean, self).__init__()
        
    def call(self, inputs):
        batch_size, patch_number, patch_size, patch_size, channels = tf.unstack(tf.shape(inputs))
        outputs = tf.reshape(inputs, shape=(-1, patch_number, patch_size, channels))
        
        centers = tf.image.resize_with_crop_or_pad(outputs, 1, 1)

        outputs = tf.math.subtract(outputs, centers)
        outputs = tf.math.pow(outputs, 2)
        outputs = tf.math.reduce_sum(outputs, axis=3)
        outputs = tf.math.pow(outputs, 0.5)
        outputs = tf.math.less_equal(outputs, tf.cast(patch_size, dtype=tf.float32))
        outputs = tf.cast(outputs, dtype=tf.int32)
        outputs = tf.reshape(outputs, shape=(-1, patch_number, patch_size, patch_size))
        
        return outputs

## Creating an array of binary values from image patches using the Manhattan colour distance function applied to the patch centre and each pixel.

According to the article:
> For each time the box $\beta_{i}$ is moved, a multidimensional analysis of colour similarity is performed for every pixel inside it. This is done by assigning the centre pixel to a vector $𝑓_{c} = 𝑟_{c}, 𝑔_{c}, 𝑏_{c}$, where $𝑟_{c}, 𝑔_{c}$ and $𝑏_{c}$ correspond to the colour intensities for each of the RGB colour channels of given pixel. The other pixels in the box are assigned to a vector $𝑓_{i} = 𝑟_{i}, 𝑔_{i}, 𝑏_{i}$ and compared to the centre pixel by calculating a colour distance $\Delta$. On the proposed approach, ... the Manhattan ($\Delta_{m}$) ...

$$
\Delta_{m} = \sum_{k} |f_{i}(k_{i}) - f_{c}(k_{c})|, k \in r, g, b
$$

In [ ]:
class Manhattan(tf.keras.layers.Layer):
    def __init__(self):
        super(Manhattan, self).__init__()
        
    def call(self, inputs):
        batch_size, patch_number, patch_size, patch_size, channels = tf.unstack(tf.shape(inputs))
        outputs = tf.reshape(inputs, shape=(-1, patch_number, patch_size, channels))
        
        centers = tf.image.resize_with_crop_or_pad(outputs, 1, 1)

        outputs = tf.math.subtract(outputs, centers)
        outputs = tf.math.abs(outputs)
        outputs = tf.math.reduce_sum(outputs, axis=3)
        outputs = tf.math.less_equal(outputs, tf.cast(patch_size, dtype=tf.float32))
        outputs = tf.cast(outputs, dtype=tf.int32)
        outputs = tf.reshape(outputs, shape=(-1, patch_number, patch_size, patch_size))
            
        
        return outputs

## Calculating probability matrices

According to the article:
> After performing this conversion for every box of every given 𝐿 scale, a structure known as probability matrix is generated. Each element of the matrix corresponds to the probability 𝑃 that 𝑚 pixels on a scale 𝐿 are labelled as 1 on each box. ... The matrix is normalized in a way that the sum of the elements in a column is equal to 1, as showed here:

$$
\sum_{m=1}^{L^2} P(m, L) = 1, \forall L
$$

In [ ]:
class Probability(tf.keras.layers.Layer):
    def __init__(self):
        super(Probability, self).__init__()
        
    def call(self, inputs):
        batch_size, patch_number, patch_size, patch_size = tf.unstack(tf.shape(inputs))
        
        outputs = tf.math.reduce_sum(inputs, axis=(2, 3))
        outputs = tf.vectorized_map(lambda image: tf.math.bincount(image, minlength=patch_size ** 2 + 1), outputs)
        outputs = tf.math.divide(outputs, patch_number)        
        
        return outputs

## Calculating fractal dimensions

According to the article:
> FD is the most common technique to evaluate the fractal properties of an image. This is a measure for evaluating the irregularity and the complexity of a fractal. To obtain local FD features from the probability
matrix, for each value of 𝐿, the FD denominated 𝐷(𝐿) is calculated according to

$$
D(L) = \sum_{m=1}^{L^2} \frac{P(m, L)}{m}
$$

In [ ]:
class FractalDimension(tf.keras.layers.Layer):
    def __init__(self):
        super(FractalDimension, self).__init__()
        
    def call(self, inputs):
        batch_size, _len = tf.unstack(tf.shape(inputs))
        numbers = tf.reshape(
            tf.concat(
                [tf.constant([1], dtype=tf.float32), tf.range(1, _len, dtype=tf.float32)], 
                axis=0
            ), 
            shape=(1, -1)
        )
        
        outputs = tf.math.divide(inputs, numbers)
        outputs = tf.math.reduce_sum(outputs, axis=1)
        
        return outputs

## Calculating lacunarity

According to the article:
> LAC is a measure complementary to FD and allows to evaluate how the space of a fractal is filled (Ivanovici & Richard, 2009). From the probability matrix, first and second-order moments are calculated with

$$
\mu(L) = \sum_{m=1}^{L^2} mP(m, L)
$$

$$
\mu^2(L) = \sum_{m=1}^{L^2} m^{2}P(m, L)
$$

> The LAC value for a scale $L$ is given by $\Lambda$(𝐿), which is obtained according to

$$
\Lambda(L) = \frac{\mu^{2}(L) - (\mu(L))^{2}}{(\mu(L))^{2}}
$$

In [ ]:
class Lacunarity(tf.keras.layers.Layer):
    def __init__(self):
        super(Lacunarity, self).__init__()
        
    def call(self, inputs):
        batch_size, _len = tf.unstack(tf.shape(inputs))
        numbers = tf.reshape(
            tf.concat(
                [tf.constant([1], dtype=tf.float32), tf.range(1, _len, dtype=tf.float32)], 
                axis=0
            ), 
            shape=(1, -1)
        )
                
        mu_first_2 = tf.math.multiply(inputs, numbers)
        mu_first_2 = tf.math.reduce_sum(mu_first_2, axis=1)
        mu_first_2 = tf.math.pow(mu_first_2, 2)

        mu_second = tf.math.pow(numbers, 2)
        mu_second = tf.math.multiply(inputs, mu_second)
        mu_second = tf.math.reduce_sum(mu_second, axis=1)

        outputs = tf.math.divide(
            tf.math.subtract(mu_second, mu_first_2),
            mu_first_2
        )
        
        return outputs

## Calculating percolation Q - the average occurrence of percolation on a scale L

According to the article:
> We can also verify whether a box $\beta_{i}$ is percolating. This can be achieved due to a property that states a percolation threshold for different types of structures. In squared matrices (digital images), this threshold has the value of $p = 0.59275$, which means that if the ratio between pixels labelled as 1 and pixels labelled as 0 is greater or equal than $p$, the matrix is considered as percolating. Let $\Omega_{i}$ be the number of pixels labelled as 1 in a box $\beta_{i}$ with size $L \times L $ , we determine whether such box is percolating according to

$$
q_{i} = 
\begin{cases}
1, & \frac{\Omega_{i}}{L^2} \ge 0.59275 \\
0, & \frac{\Omega_{i}}{L^2} < 0.59275
\end{cases}
$$

> This results in a binary value for $q_{i}$, wherein 1 indicates that thebox is percolating. The feature $Q(L)$ regards the average occurrence of percolation on a scale $L$ and can be obtained by

$$
Q(L) = \frac{\sum_{i=1}^{T(L)} q_{i}}{T(L)}
$$

In [ ]:
class PercolationQ(tf.keras.layers.Layer):
    def __init__(self, threshold=0.59275):
        super(PercolationQ, self).__init__()
        
        self.threshold = threshold
        
    def call(self, inputs):
        batch_size, patch_number, patch_size, patch_size = tf.unstack(tf.shape(inputs))
        
        outputs = tf.math.reduce_sum(inputs, axis=(2, 3))
        outputs = tf.math.divide(outputs, patch_size ** 2)
        outputs = tf.math.greater_equal(outputs, self.threshold)
        outputs = tf.cast(outputs, dtype=tf.float32)
        outputs = tf.math.reduce_mean(outputs, axis=1)
        
        return outputs

## Clustering values in binarized patches 

The next two layers, which calculate percolation C and M, work with value clusters. We clustorize values in a separate layer to speed up calculations.

In [ ]:
class Clusterize(tf.keras.layers.Layer):
    def __init__(self):
        super(Clusterize, self).__init__()
        
    def call(self, inputs):
        batch_size, patch_number, patch_size, patch_size = tf.unstack(tf.shape(inputs))
        
        outputs = tf.reshape(inputs, shape=(-1, patch_size, patch_size))
        outputs = tfa.image.connected_components(outputs)
        outputs = tf.reshape(outputs, shape=(-1, patch_number, patch_size, patch_size))
        
        return outputs

## Calculating percolation C - the average number of clusters per box on a scale L

According to the article:
> Let $c_{i}$ be the number of clusters on a box $\beta_{i}$, the feature $C(L)$ that represents the average number of clusters per box on a scale $L$ is given by

$$
C(L) = \frac{\sum_{i=1}^{T(L)} c_{i}}{T(L)}
$$

In [ ]:
class PercolationC(tf.keras.layers.Layer):
    def __init__(self):
        super(PercolationC, self).__init__()
        
    def call(self, inputs):
        outputs = tf.cast(inputs, dtype=tf.float32)
        outputs = tf.math.reduce_max(outputs, axis=(2, 3))
        outputs = tf.math.reduce_mean(outputs, axis=1)
        
        return outputs

## Calculating percolation M - the average coverage area of the largest cluster on a scale L

According to the article:
>Another feature that can be obtained is the average coverage area of the largest cluster in a box and is given by $M(L)$. Let $m_{i}$ be the size in pixels of the largest cluster of the box $\beta_{i}$. The feature $M(L)$ is givenaccording to

$$
M(L) = \frac{\sum_{i=1}^{T(L)} \frac{m_{i}}{L^2}}{T(L)}
$$

In [ ]:
class PercolationM(tf.keras.layers.Layer):
    def __init__(self):
        super(PercolationM, self).__init__()
        
    def call(self, inputs):
        batch_size, patch_number, patch_size, patch_size = tf.unstack(tf.shape(inputs))
        
        outputs = tf.reshape(inputs, shape=(-1, patch_number, patch_size ** 2))
        outputs = tf.map_fn(lambda image: tf.math.reduce_max(tf.math.bincount(image)), outputs)
        outputs = tf.cast(outputs, dtype=tf.float32)
        
        return outputs

## Assembling fractal features into an image channel

According to the article:
> To serve as input for the incoming CNN classification, the feature vectors generated on the previous layers of the network must be converted into feature matrices. To do so, the 100 features obtained by each distance $\Delta$ are rearranged as a $10 \times 10 \times 10$ matrix. The matrices generated by $\Delta_{h}$, $\Delta_{e}$ and $\Delta_{m}$ correspond to the R, G and B colour channels, respectively. ... Since each of the functions $C(L), Q(L), M(L), \Lambda(L)$ and $D(L)$, obtained from a specific $\Delta$, generate 20 features, each function is fit exactly into 2 columns of the matrix.

>Since each of the functions $C(L), Q(L), M(L), \Lambda(L)$ and $D(L)$, obtained from a specific $\Delta$, generate 20 features, each function is fit exactly into 2 columns of the matrix.

In [ ]:
class AssembleChannel(tf.keras.layers.Layer):
    def __init__(self):
        super(AssembleChannel, self).__init__()
        
    def call(self, inputs):
        fractal_dimension = tf.convert_to_tensor(inputs[0])
        fractal_dimension = tf.transpose(fractal_dimension, perm=(1, 0))
        
        lacunarity = tf.convert_to_tensor(inputs[1])
        lacunarity = tf.transpose(lacunarity, perm=(1, 0))
        
        percolation_q = tf.convert_to_tensor(inputs[2])
        percolation_q = tf.transpose(percolation_q, perm=(1, 0))
        
        percolation_c = tf.convert_to_tensor(inputs[3])
        percolation_c = tf.transpose(percolation_c, perm=(1, 0))
        
        percolation_m = tf.convert_to_tensor(inputs[4])
        percolation_m = tf.transpose(percolation_m, perm=(1, 0))
        
        outputs = tf.concat([
            percolation_c,
            percolation_q,
            percolation_m,
            lacunarity,
            fractal_dimension
        ], axis=1)
        outputs = tf.reshape(outputs, shape=(-1, 10, 10))
        
        return outputs

## Organising fractal feature extraction into layers

We move feature extraction to layers to simplify and clarify the code.

### based on Chebyshev distance

In [ ]:
class ChebyshevFeatures(tf.keras.layers.Layer):
    def __init__(self):
        super(ChebyshevFeatures, self).__init__()
        
        self.chebyshev = Chebyshev()
        self.probability = Probability()
        self.clusterize = Clusterize()
        
        self.fractal_dimension = FractalDimension()
        self.lacunarity = Lacunarity()
        self.percolation_q = PercolationQ()
        self.percolation_c = PercolationC()
        self.percolation_m = PercolationM()
        
        self.assemble_channel = AssembleChannel()
        
    def call(self, inputs):
        chebyshevs = [self.chebyshev(i) for i in inputs]
        
        probability = [self.probability(ch) for ch in chebyshevs]
        cluster = [self.clusterize(ch) for ch in chebyshevs]
        
        fractal_dimension = [self.fractal_dimension(ch) for ch in probability]
        lacunarity = [self.lacunarity(ch) for ch in probability]        
        percolation_q = [self.percolation_q(ch) for ch in chebyshevs]
        percolation_c = [self.percolation_c(ch) for ch in cluster]
        percolation_m = [self.percolation_m(ch) for ch in cluster]
        
        features = self.assemble_channel([
            fractal_dimension,
            lacunarity,
            percolation_q,
            percolation_c,
            percolation_m
        ])
        
        return features

### based on Euclidean distance

In [ ]:
class EuclideanFeatures(tf.keras.layers.Layer):
    def __init__(self):
        super(EuclideanFeatures, self).__init__()
        
        self.euclidean = Euclidean()
        self.probability = Probability()
        self.clusterize = Clusterize()
        
        self.fractal_dimension = FractalDimension()
        self.lacunarity = Lacunarity()
        self.percolation_q = PercolationQ()
        self.percolation_c = PercolationC()
        self.percolation_m = PercolationM()
        
        self.assemble_channel = AssembleChannel()
        
    def call(self, inputs):
        euclideans = [self.euclidean(i) for i in inputs]
        
        probability = [self.probability(eu) for eu in euclideans]
        cluster = [self.clusterize(eu) for eu in euclideans]
        
        fractal_dimension = [self.fractal_dimension(eu) for eu in probability]
        lacunarity = [self.lacunarity(eu) for eu in probability]        
        percolation_q = [self.percolation_q(eu) for eu in euclideans]
        percolation_c = [self.percolation_c(eu) for eu in cluster]
        percolation_m = [self.percolation_m(eu) for eu in cluster]
        
        features = self.assemble_channel([
            fractal_dimension,
            lacunarity,
            percolation_q,
            percolation_c,
            percolation_m
        ])
        
        return features

### based on Manhattan distance

In [ ]:
class ManhattanFeatures(tf.keras.layers.Layer):
    def __init__(self):
        super(ManhattanFeatures, self).__init__()
        
        self.manhattan = Manhattan()
        self.probability = Probability()
        self.clusterize = Clusterize()
        
        self.fractal_dimension = FractalDimension()
        self.lacunarity = Lacunarity()
        self.percolation_q = PercolationQ()
        self.percolation_c = PercolationC()
        self.percolation_m = PercolationM()
        
        self.assemble_channel = AssembleChannel()
        
    def call(self, inputs):
        manhattans = [self.manhattan(i) for i in inputs]
        
        probability = [self.probability(mh) for mh in manhattans]
        cluster = [self.clusterize(mh) for mh in manhattans]
        
        fractal_dimension = [self.fractal_dimension(mh) for mh in probability]
        lacunarity = [self.lacunarity(mh) for mh in probability]        
        percolation_q = [self.percolation_q(mh) for mh in manhattans]
        percolation_c = [self.percolation_c(mh) for mh in cluster]
        percolation_m = [self.percolation_m(mh) for mh in cluster]
        
        features = self.assemble_channel([
            fractal_dimension,
            lacunarity,
            percolation_q,
            percolation_c,
            percolation_m
        ])
        
        return features

## Assembling fractal features into images

We assemble fractal features into images, such that each set of fractal features corresponds to a colour channel (R, G, B).

In [ ]:
class AssembleImage(tf.keras.layers.Layer):
    def __init__(self):
        super(AssembleImage, self).__init__()
        
    def call(self, inputs):
        outputs = tf.stack(inputs)
        outputs = tf.transpose(outputs, perm=(1, 2, 3, 0))
        
        return outputs

## Organising the fractal feature extraction layers into the single, fractal image layer

To further simplify the code, we will gather the fractal feature extraction into the single layer, which generates artificial fractal image.

In [ ]:
class FractalImage(tf.keras.layers.Layer):
    def __init__(self):
        super(FractalImage, self).__init__()
        
        self.patchifies = [Patchify(patch_size) for patch_size in range(3, 41 + 1, 2)]
        
        self.chebyshev_features = ChebyshevFeatures()
        self.euclidean_features = EuclideanFeatures()
        self.manhattan_features = ManhattanFeatures()
        
        self.assemble_image = AssembleImage()
        
    def call(self, inputs):
        patchifies = [patchify(inputs) for patchify in self.patchifies]
        
        chebyshev_features = self.chebyshev_features(patchifies)
        euclidean_features = self.euclidean_features(patchifies)
        manhattan_features = self.manhattan_features(patchifies)
        
        outputs = self.assemble_image([
            chebyshev_features,
            euclidean_features,
            manhattan_features
        ])
        
        return outputs

# Assembling the fractal neural network

So, here we are assembling the fractal neural network from the pieces mentioned above.

In [ ]:
class FractalNeuralNetwork(tf.keras.Model):
    TARGET_WIDTH, TARGET_HEIGHT = 299, 299
    
    def __init__(self, class_number):
        super(FractalNeuralNetwork, self).__init__()
        
        self.fractal_image = FractalImage()
        self.resize = tf.keras.layers.Resizing(
            width=self.TARGET_WIDTH,
            height=self.TARGET_HEIGHT
        )
        self.rescale = tf.keras.layers.Rescaling(scale=1./255)
        
        self.original_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(299, 299, 3),
            pooling='avg'
        )
        for layer in self.original_model.layers:
            if layer.name in ['conv_7b', 'block8_10_conv']:
                layer.trainable = True
            else:
                layer.trainable = False
                
        self.fractal_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(299, 299, 3),
            pooling='avg'
        )
        for layer in self.fractal_model.layers:
            if layer.name in ['conv_7b', 'block8_10_conv']:
                layer.trainable = True
            else:
                layer.trainable = False
                
        self.combine = tf.keras.layers.Concatenate()
        self.score = tf.keras.layers.Dense(class_number, activation='softmax')
        
    def call(self, inputs):
        fractal_outputs = self.fractal_image(inputs)
        fractal_outputs = self.resize(fractal_outputs)
        fractal_outputs = self.rescale(fractal_outputs)
        fractal_outputs = self.fractal_model(fractal_outputs)
        
        original_outputs = self.rescale(inputs)
        original_outputs = self.original_model(original_outputs)
        
        outputs = self.combine([fractal_outputs, original_outputs])
        outputs = self.score(outputs)
        
        return outputs

# Data research

We load the __metadata.csv__, which contains information about each image.

In [ ]:
metadata = pd.read_csv(f"{os.environ['SCRATCH']}/isic-archive/metadata.csv")
metadata

__Note__:
As we discovered, there are invalid image files. Let's remove those from the metadata.

In [ ]:
metadata = metadata.drop(metadata[metadata['isic_id'].isin(['ISIC_0060052', 'ISIC_0029842'])].index)

Two columns are particularly important for research: __benign_malignant__ - indicates whether the lesion is benign or malignant without any particular diagnosis, and __diagnosis__ column. 

We break down the data by __benign_malignant__ and __diagnosis__ to see, what is the balance there.

In [ ]:
diagnosis_breaddown_metadata = metadata.groupby(['diagnosis'], dropna=False, as_index=False).size()

In [ ]:
benign_malignant_breaddown_metadata = metadata.groupby(['benign_malignant'], dropna=False, as_index=False).size()

In [ ]:
%matplotlib inline

plt.rcParams['axes.titley'] = -0.7

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(23,5))

# diagnosis plot
data = diagnosis_breaddown_metadata['size']
keys = diagnosis_breaddown_metadata['diagnosis']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax1.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))
ax1.set_title('Pic.3. Metadata breakdown by diagnosis')
ax1.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

# diagnosis plot
data = benign_malignant_breaddown_metadata['size']
keys = benign_malignant_breaddown_metadata['benign_malignant']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax2.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))

ax2.set_title('Pic. 4. Metadata breakdown by malignancy')
ax2.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

<a id='metadata_breakdowns'></a>
From the first pie chart, we see, that __39.28%__ samples don't have a diagnosis. We think, that the fact somehow correlates with the lesions' malignancy and the way the diagnosis' were obtained.

In [ ]:
no_diagnosis_metadata = metadata[pd.isnull(metadata['diagnosis'])]

In [ ]:
benign_malignant_breaddown_no_diagnosis_metadata = no_diagnosis_metadata.groupby(['benign_malignant'], dropna=False, as_index=False).size()

In [ ]:
diagnosis_confirm_type_breaddown_no_diagnosis_metadata = no_diagnosis_metadata.groupby(['diagnosis_confirm_type'], dropna=False, as_index=False).size()

In [ ]:
%matplotlib inline

plt.rcParams['axes.titley'] = -0.7

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(23,5))

# diagnosis plot
data = benign_malignant_breaddown_no_diagnosis_metadata['size']
keys = benign_malignant_breaddown_no_diagnosis_metadata['benign_malignant']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax1.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))
ax1.set_title('Pic. 5. Metadata with no dignosis breakdown by malignancy')
ax1.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

# diagnosis plot
data = diagnosis_confirm_type_breaddown_no_diagnosis_metadata['size']
keys = diagnosis_confirm_type_breaddown_no_diagnosis_metadata['diagnosis_confirm_type']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax2.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))

ax2.set_title('Pic. 6. Metadata with no dignosis breakdown by diagnosis confirmation type')
ax2.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

__99.72%__ of samples without a diagnosis are benign and only __1.52%__ are consirmed with histopathology research. 

Probably, because excision wasn't done, the hospital protocol allowed not to put exact diagnosis. But imaging researches were enough to conclude that the lesions were benign.

Let's see, what kind of research is made with the samples, which have diagnosis.

In [ ]:
yes_diagnosis_metadata = metadata[pd.notnull(metadata['diagnosis'])]

In [ ]:
benign_malignant_breaddown_yes_diagnosis_metadata = yes_diagnosis_metadata.groupby(['benign_malignant'], dropna=False, as_index=False).size()

In [ ]:
diagnosis_confirm_type_breaddown_yes_diagnosis_metadata = yes_diagnosis_metadata.groupby(['diagnosis_confirm_type'], dropna=False, as_index=False).size()

In [ ]:
%matplotlib inline

plt.rcParams['axes.titley'] = -0.7

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(23,5))

# diagnosis plot
data = benign_malignant_breaddown_yes_diagnosis_metadata['size']
keys = benign_malignant_breaddown_yes_diagnosis_metadata['benign_malignant']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax1.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))
ax1.set_title('Pic. 7. Metadata with no dignosis breakdown by malignancy')
ax1.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

# diagnosis plot
data = diagnosis_confirm_type_breaddown_yes_diagnosis_metadata['size']
keys = diagnosis_confirm_type_breaddown_yes_diagnosis_metadata['diagnosis_confirm_type']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax2.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))

ax2.set_title('Pic. 8. Metadata with no dignosis breakdown by diagnosis confirmation type')
ax2.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

Out of the samples with diagnosis, much more of them were confirmed with a histopathological research - __48.18%__. But, and that's important, if a sample has a diagnosis, it doesn't mean, that it was confirmed with a histopathological research.

Let's go back to the [Pic. 4. Breakdown by malignancy](#metadata_breakdowns) . __10.92%__ of samples have unknown malignancy, so we will study it through diagnosis and other columns.

In [ ]:
no_benign_malignant_metadata = metadata[pd.isnull(metadata['benign_malignant'])]

In [ ]:
diagnosis_breakdown_no_benign_malignant_metadata = no_benign_malignant_metadata.groupby(['diagnosis'], dropna=False, as_index=False).size()

In [ ]:
diagnosis_confirm_type_no_benign_malignant_metadata = no_benign_malignant_metadata.groupby(['diagnosis_confirm_type'], dropna=False, as_index=False).size()

In [ ]:
%matplotlib inline

plt.rcParams['axes.titley'] = -0.7

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(23,5))

# diagnosis plot
data = diagnosis_breakdown_no_benign_malignant_metadata['size']
keys = diagnosis_breakdown_no_benign_malignant_metadata['diagnosis']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax1.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))
ax1.set_title('Pic. 9. Metadata with no malignancy breakdown by diagnosis')
ax1.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

# diagnosis plot
data = diagnosis_confirm_type_no_benign_malignant_metadata['size']
keys = diagnosis_confirm_type_no_benign_malignant_metadata['diagnosis_confirm_type']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax2.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))

ax2.set_title('Pic. 10. Metadata with no malignancy breakdown by diagnosis confirmation type')
ax2.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

For the most cases, the issues is with incorrectly filled data:
- [Basal-cell carcinoma](https://en.wikipedia.org/wiki/Basal-cell_carcinoma) - malignant
- Pigmented benign ketosis - benign
- [Seborrheic keratosis](https://en.wikipedia.org/wiki/Seborrheic_keratosis) - benign
- [Actinic keratosis](https://en.wikipedia.org/wiki/Actinic_keratosis) - precancerous, untreated lesions have up to a 20% risk of progression to squamous cell carcinoma, for our purposes, we'll consider it malignant
- [Squamous cell carcinoma](https://en.wikipedia.org/wiki/Squamous_cell_carcinoma) - malignant
- [Vascular lesion](https://en.wikipedia.org/wiki/Vascular_anomaly) - benign
- [Dermatofibroma](https://en.wikipedia.org/wiki/Dermatofibroma) - benign
- [Solar lentigo](https://en.wikipedia.org/wiki/Liver_spot) - benign

Let's fix the data

In [ ]:
metadata.loc[metadata['diagnosis'] == 'basal cell carcinoma', 'benign_malignant']  = 'malignant'
metadata.loc[metadata['diagnosis'] == 'pigmented benign keratosis', 'benign_malignant']  = 'benign'
metadata.loc[metadata['diagnosis'] == 'seborrheic keratosis', 'benign_malignant']  = 'benign'
metadata.loc[metadata['diagnosis'] == 'actinic keratosis', 'benign_malignant']  = 'malignant'
metadata.loc[metadata['diagnosis'] == 'squamous cell carcinoma', 'benign_malignant']  = 'malignant'
metadata.loc[metadata['diagnosis'] == 'vascular lesion', 'benign_malignant']  = 'benign'
metadata.loc[metadata['diagnosis'] == 'dermatofibroma', 'benign_malignant']  = 'benign'
metadata.loc[metadata['diagnosis'] == 'solar lentigo', 'benign_malignant']  = 'benign'

Returning to samples without a diagnosis, we have two ways out of this:
- remove the samples
- introduce a new "label"

We don't know much about those samples except that they contain all kind of benign skin conditions. So, for example, there will be __solar lentigo__ samples in the __solar lentigo__ subset and the undiagnosed subset. That will may lead to classifying __solar lentigo__ as a part of undiagnosed subset. That's why, it's better to exclude the undiagnosed samples from training.

In addition to that, the dataset is tilted toward benign skin conditions, and almost all undiagnosed samples represent benign skin conditions. In that way, we not only remove confusing data but perform undersampling to balance out the dataset.

In [ ]:
metadata = metadata.drop(metadata[pd.isnull(metadata['diagnosis'])].index)

Later on we will need to split the dataset into training, validation and testing. To do that, all diagnoses should have sufficient amount of  samples.

In [ ]:
diagnosis_numbers = metadata.groupby(['diagnosis'], dropna=False, as_index=False).size().sort_values(by='size')
diagnosis_numbers

We'll remove the samples, which are not part of the 10 most common diagnosis categories.

In [ ]:
rare_diagnosis = diagnosis_numbers[:-10]['diagnosis']
rare_diagnosis

In [ ]:
metadata = metadata.drop(metadata[metadata['diagnosis'].isin(rare_diagnosis)].index)
metadata = metadata.drop(metadata[~metadata['benign_malignant'].isin(['benign', 'malignant'])].index)

Let's look at the breakdown now.

In [ ]:
diagnosis_breaddown_metadata = metadata.groupby(['diagnosis'], dropna=False, as_index=False).size()

In [ ]:
benign_malignant_breaddown_metadata = metadata.groupby(['benign_malignant'], dropna=False, as_index=False).size()

In [ ]:
%matplotlib inline

plt.rcParams['axes.titley'] = -0.7

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(23,5))

# diagnosis plot
data = diagnosis_breaddown_metadata['size']
keys = diagnosis_breaddown_metadata['diagnosis']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
shuffle(palette_color)
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax1.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))
ax1.set_title('Pic.11. Metadata breakdown by diagnosis after preprocessing')
ax1.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

# diagnosis plot
data = benign_malignant_breaddown_metadata['size']
keys = benign_malignant_breaddown_metadata['benign_malignant']
palette_color = ['#333745', '#E63462', '#FE5F55', '#C7EFCF', '#C7EFCF', '#D5573B', '#885053', '#777DA7', '#94C9A9', '#C6ECAE', '#3E92CC', '#2A628F', '#13293D', '#16324F', '#18435A', '#F4E04D', '#FCFCFC', '#54F2F2', '#5EB1BF', '#042A2B', '#EAC435', '#345995', '#03CEA4', '#FB4D3D', '#CA1551', '#89023E', '#CC7178', '#FFD9DA', '#C7D9B7']
percentages = 100. * data / data.sum()

# plotting data on chart
patches, texts = ax2.pie(data, colors=palette_color, startangle=90, radius=2.5)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(keys, percentages)]
patches, labels, dummy =  zip(*sorted(zip(patches, labels, data),
                                      key=lambda x: x[2],
                                      reverse=True))

ax2.set_title('Pic. 12. Metadata breakdown by malignancy after preprocessing')
ax2.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=10)

The dataset is still imbalanced, so we'll calculate class weights to use them in training.

In [ ]:
class_weights = metadata.groupby(['diagnosis'], dropna=False, as_index=False).size()
class_weights.loc[:, 'size'] =  1 / class_weights.loc[:, 'size']
class_weights.loc[:, 'size'] *= ( len(metadata) / 10 )
class_weights

In [ ]:
class_weights = dict(enumerate(class_weights['size']))

We transform ISIC ids to filenames. 

In [ ]:
metadata['isic_id'] += '.jpg'

We need to extract testing files from the dataset.

In [ ]:
rest_set, testing_set = train_test_split(metadata, test_size=0.1)

Finally, we can load the images.

In [ ]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=180,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.2, 1.5),
    validation_split=0.2
)

training_set = generator.flow_from_dataframe(
    rest_set,
    directory=f"{os.environ['SCRATCH']}/isic-archive",
    x_col='isic_id',
    y_col='diagnosis',
    
    target_size=(299, 299), 
    batch_size=32, 
    class_mode='categorical', 
    subset='training'
)

validation_set = generator.flow_from_dataframe(
    rest_set,
    directory=f"{os.environ['SCRATCH']}/isic-archive",
    x_col='isic_id',
    y_col='diagnosis',
    
    target_size=(299, 299), 
    batch_size=32, 
    class_mode='categorical', 
    subset='validation'
)

testing_set = generator.flow_from_dataframe(
    testing_set,
    directory=f"{os.environ['SCRATCH']}/isic-archive",
    x_col='isic_id',
    y_col='diagnosis',
    
    target_size=(299, 299), 
    batch_size=32, 
    class_mode='categorical'
)

In [ ]:
CLASS_NUMBER = len(training_set.class_indices)

## Data source

As a data source, we use the ISIC Archive.

The ISIC Archive is an open source platform with publicly available images of skin lesions under Creative Commons licenses. The images are associated with ground-truth diagnoses and other clinical metadata. Images can be queried using faceted search and downloaded individually or in batches. The initial focus of the archive has been on dermoscopy images of individual skin lesions, as these images are inherently standardized by the use of a specialized acquisition device and devoid of many of the privacy challenges associated with clinical images. To date, the images have been provided by specialized melanoma centers from around the world. The archive is designed to accept contributions from new sources under the Terms of Use and welcomes new contributors. There are ongoing efforts to supplement the dermoscopy images in the archive with close-up clinical images and a broader representation of skin types. The images in the Archive are used to support educational efforts through linkage with Dermoscopedia and are used for Grand Challenges and Live Challenges to engage the computer science community for the development of diagnostic AI.

For more information, go to [ISIC Archive web site](https://www.isic-archive.com/)

# Model training

## Preparing TensorFlow callbacks

For our convenience, we create a few TensorFlow callbacks.

### The EarlyStopping callback

This callback stops training when the metrics (e.g. validation loss) are not improving,

In [ ]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    min_delta=0.01, 
    patience=5, 
    restore_best_weights=True
)

### The ModelCheckpoint callback

This callback saves the model with the best metrics during training.

In [ ]:
checkpoint_path = 'checkpoints/model.ckpt'

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    save_freq='epoch',
    mode='auto'
)

## Define the distribution strategy

In [ ]:
strategy = tf.distribute.MirroredStrategy()

## Actual training

In [ ]:
with strategy.scope():
    model = FractalNeuralNetwork(class_number=CLASS_NUMBER)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    training_set, 
    validation_data=validation_set, 
    epochs=30,
    callbacks=[
        early_stop_callback,
        checkpoint_callback
    ]
)

# Model validation

## Loading the model from the checkpoint

In [ ]:
model = FractalNeuralNetwork(class_number=CLASS_NUMBER)

In [ ]:
model.load_weights(checkpoint_path)

## Making diagnoses

In [ ]:
true_labels = np.concatenate([testing_set[i][1] for i in range(len(testing_set))], axis=0)

In [ ]:
predicted_labels = model.predict(testing_set)

## Plot the ROC Curve

In [ ]:
fpr = dict()
tpr = dict()
auc_metric = dict()

diagnosis_index_dict = {v: k for k, v in testing_set.class_indices.items()}

for i in range(len(diagnosis_index_dict)):
    diagnosis = diagnosis_index_dict[i]
    fpr[diagnosis], tpr[diagnosis], _ = roc_curve(true_labels[:, i], predicted_labels[:, i])
    auc_metric[diagnosis] = auc(fpr[diagnosis], tpr[diagnosis])

In [ ]:
%matplotlib inline 
for diagnosis in testing_set.class_indices:
    plt.plot(fpr[diagnosis], tpr[diagnosis], label=diagnosis)
    
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

## Show AUC

In [ ]:
auc_metric

# Model explanation

Now we want some explanations from out model.

We load an image with melanoma

In [ ]:
melanoma_im_path = f"{os.environ['SCRATCH']}/isic-archive/ISIC_0000031.jpg"

In [ ]:
display(Image(melanoma_im_path, width=600))

Pic. 14. Image with melanoma.

In [ ]:
def get_img_array(img_path, size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

In [ ]:
img_array = tf.keras.applications.inception_resnet_v2.preprocess_input(get_img_array(melanoma_im_path, size=(299, 299)))

and plot Grad-CAM over the melanoma image,

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
heatmap = make_gradcam_heatmap(img_array, model.original_model, 'conv_7b')

In [ ]:
def display_gradcam(img_path, heatmap, alpha=0.4):
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)
    
    heatmap = np.uint8(255 * heatmap)
    
    jet = cm.get_cmap("jet")
    
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]
    
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)
    
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    superimposed_img = superimposed_img.resize((np.array(superimposed_img.size) * 600 / 1504).astype(int))


    display(superimposed_img)

In [ ]:
display_gradcam(melanoma_im_path, heatmap)

Pic. 15. Grad-CAM image of the melanoma.